## 0. 简介

本 note 中主要实现神经网络中常见的函数，所有函数的实现均考虑向量计算，包括：

1. `sigmoid(x)` - 处理格式张量
2. `relu(x)` - 处理任意格式张量
3. `softmax(x)` - 向量数据或者向量数据组成的 mini-batch 数据
4. `cross_entropy_entry(y, t)` - - 向量数据或者向量数据组成的 mini-batch 数据，其中`y`是预测标签，`t`是实际标签

## 1. sigmoid

实现如下：

In [3]:
def sigmoid(x):
    '''
    Parameters
    ----------
    x : 任意维度的张量
    
    Return
    ------
    施加 sigmoid 计算后的 x
    '''
    
    return 1 / 1 + np.exp(-x)

## 2. relu

`relu()` 的实现主要是将其行为用 `maximum(0, x)` 函数来实现

In [4]:
def relu(x):
    '''
    Parameters
    ----------
    x : 任意维度的张量
    
    Return
    ------
    施加 relu 计算后的 x
    '''
    return np.maximum(0, x)  # 取 x 和 0 的各个位置上较大的元素返回

## 3. softmax

`softmax` 的实现要点如下：

1. 数值计算的问题，即计算 `exp(a)`时，如果`a` 过大，会出现 `inf` 的情况，两个 `inf` 做除法会出现不确定的情况，而 $C\times \exp(x) = \exp(x + \ln{C})$，所以可以先对 `x` 中的元素先减去对应的最大值避免 `inf` 的情况出现
2. 根据输入数据是向量还是 mini-batch 对应的二维矩阵，分别进行处理

In [5]:
def softmax(x):
    '''
    Paramters
    ---------
    x : 向量或者 mini-batch 下的矩阵，每一行对应一条数据
    
    Return
    ------
    施加 softmax 计算后的 x
    '''
    
    ## 先检测是否是 mini-batch 的情况
    ## mini-batch 需要每一行单独处理
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True) # 各条数据分别减去各自所有元素中的最大值
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
        
    elif x.ndim == 1:  # 单条数据
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))
        
    return x

## 4. cross_entropy_error

交叉熵误差的实现要点在于：

0. 需要区分训练数据中 `t` 是 one-hot 形式给出还是直接以单独的标签数字给出，相比标签数据，one-hot 格式需要做一步解码索引的操作来确定具体的标签值，最后统一转换为 one-hot 格式来处理

1. 因为 `y` 来自于上一层类似于 `softmax()` 的输出结果，所以 `y` 一定不是一个单一的数据标签

2. 如果 `t.size == y.size` 根据 1，`t` 一定是 one-hot 格式

3. 因为 one-hot 格式的特殊性，只有 t 中为 1 的位置对应的元素才会最终算入交叉熵误差中，所以可以用索引 y 在正确输出位置的值来计算交叉熵，即 `y[np.arange(batch_size), t]`

3. 为了保证 `log(x)` 中的 $x$ 不接近于 $0$，可以加上一个足够小的值，即 `log(x + 1e-7)`，一般情况下 `1e-7` 会在浮点数的近似表示中被“吸收”

4. 输入数据默认按 mini-batch 处理，其中每一行代表一条数据，因为单条数据属于 mini-batch 的特例，需要将其格式转换为 mini-batch 的形式

In [1]:
def corss_entropy_error(y, t):
    '''
    Paramters
    ---------
    y : 预测标签
    t : 正确标签
    
    Return
    ------
    交叉熵误差
    '''
    
    ## 如果是单条数据，也转换为 mini-batch 形式
    if y.ndim == 1:  # 标签格式，转换为行向量形式
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)

    ## 数据是 one-hot 形式
    if t.size == y.size:  # t 是 one-hot 格式
        t = t.argmax(axis=1)  # 取回 t 的标签，因为只有标签处的值才是 1，其他处为 0
    
    batch_size = y.shape[0]
    
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size